# Tutorial & Widget Experiments

> This notebook is primarily being used to experiment with how to use the `pymbe` widgets.
>
> It should be fairly "clean" until the `Scratch Pad` section.
>

In the future, this notebook will be cleaned up and renamed "Tutorial".

## 1. Import `pymbe` and create a new user interface

In [ ]:
import pymbe.api as pm

In [ ]:
pm.UI.new(host_url="http://sysml2.intercax.com")

## 2. Use the widget
...or automatically load the `Kerbal` model using the cell below by:

1. Grabbing the individual widgets
> and view the log for the Diagram of the Labelled Property Graph

In [ ]:
ui, *_ = _.children
tree, inspector, m1_diagram, m0_diagram = ui.children

2. load a small model from a file or the big model from the internet

In [ ]:
client.project_selector.options

In [ ]:
USE_FILE = False

PROJECT_NAME = "12b-Allocation"

if USE_FILE:
    tree.model = pm.Model.load_from_file(FIXTURES / f"{PROJECT_NAME}.json")
else:
    client = tree.api_client
    project_names = [name for name in client.project_selector.options if name.startswith(PROJECT_NAME)]
    if project_names:
        *_, last_project = project_names
        client.project_selector.label = last_project
        client.download_model.click()

In [ ]:
m1_diagram.elk_diagram.toolbar.refresh_diagram.click()

# Intepretation Diagram

1. [ ] Get the id of the interpretation, which is a connection usage, and retrieve that element
2. [ ] Look at the `connectorEnd[:2]` and retrieve interpretations for them

# Add all implied edges

In [ ]:
from pymbe.interpretation.calc_dependencies import *
from pymbe.interpretation.interp_playbooks import random_generator_playbook

instances = random_generator_playbook(m1=lpg)
pairs = generate_execution_order(lpg=lpg, instance_dict=instances)

In [ ]:
from pymbe.graph.calc_lpg import *

group = CalculationGroup(lpg.get_projection("Expression Inferred Graph"), instances, pairs)

In [ ]:
group.solve_graph(lpg)

In [ ]:
instances["1e5a0ed7-8b41-4ab4-a433-8f7eedd75833"]

In [ ]:
lpg.elements_by_id = {**lpg.elements_by_id, **lpg.get_implied_elements()}

...or you can directly adapt the graph and invert edges

In [ ]:
import data_loader

In [ ]:
from pymbe.client import APIClient
from pathlib import Path


def kerbal_model_loaded_client() -> APIClient:
    helper_client = APIClient()
    helper_client.model = pm.Model.load_from_file(Path(data_loader.__file__).parent / "data" / "Kerbal" / "elements.json")
    return helper_client

model = kerbal_model_loaded_client()

In [ ]:
model

In [ ]:
os.listdir()

In [ ]:
from tests.interpretation.test_playbook_phases import *

In [ ]:
import os

os.chdir("tests/")

In [ ]:
from interpretation.test_playbook_phases import kerbal_client

test_client = kerbal_client()

In [ ]:
client.elements_by_type

# Add RDF Capabilities with `ipyradiant`

In [ ]:
import traitlets as trt
from ipyradiant.visualization import CytoscapeViewer, InteractiveViewer
from pymbe.graph.rdf import SysML2RDFGraph

In [ ]:
rdf = SysML2RDFGraph()
trt.link((lpg, "elements_by_id"), (rdf, "elements_by_id"))

In [ ]:
rdf.graph.namespace_manager.bind("purl", 'http://purl.org/dc/terms/identifier')

In [ ]:
InteractiveViewer(rdf_graph=rdf.graph)

In [ ]:
from rdflib import Graph

g = Graph()
triples = list(rdf.graph.triples([None, None, None]))[1000:]
g.addN([[*triples, "SysMLv2"] for triple in triples])

In [ ]:
%debug

In [ ]:
import ipywidgets as ipyw

file_uploader = ipyw.FileUpload(
    accept='*.json',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,   # True to accept multiple files upload else False
)
file_uploader

In [ ]:
import requests

resp = requests.get(client.elements_url)


In [ ]:
import json
from pathlib import Path

projects = {
    name.split(" (2021")[0]: project_id
    for name, project_id in client.project_selector.options.items()
}
projects


for name, project_id in projects.items():
    try:
        client.selected_project = project_id
        data = client._get_elements_from_server()
        with Path(f"{name}.json") as f:
            f.write_text(json.dumps(data))
        print(f"Downloaded {name}, commit: {client.selected_commit}")
    except:
        print(f"Failed to download: {name}...")

In [ ]:
for id_, el in rdf.elements_by_id.items():
    context = el.get("@context")
    if context:
        print(f"{id_}: has context ({context})")
    else:
        print(f"{id_} has no context")

# Interpretation

> This is where we are refactoring the $M_0$ interpretation...

In [ ]:
lpg.sysml_projections["Complete"]["description"]

In [ ]:
def safe_get(data: dict, *keys, default=None):
    for key in keys:
        if key not in data:
            return default
        data = data[key]
    return data

safe_get(lpg.elements_by_id, "cb4386fe-67d9-4743-995f-5d4a4162e26e", "owner", "@id")

In [ ]:
UNSPECIFIED = object()

class SafeDict(dict):

    def __init__(self, *args, default=None, **kwargs):
        self.__default__ = default or {}
        super().__init__(*args, **kwargs)
        for key, value in self.items():
            self[key] = SafeDict.__convert(value)

    def __setitem__(self, k, v):
        dict.__setitem__(self, k, SafeDict.__convert(v))

    def get(self, item, default=None):
        return super().get(item, default) or self.__default__

    @staticmethod
    def __convert(o):
        """
        Recursively convert `dict` objects in `dict`, `list`, `set`, and
        `tuple` objects to `attrdict` objects.
        """
        if isinstance(o, dict):
            o = SafeDict(o)
        elif isinstance(o, list):
            o = list(SafeDict.__convert(v) for v in o)
        elif isinstance(o, set):
            o = set(SafeDict.__convert(v) for v in o)
        elif isinstance(o, tuple):
            o = tuple(SafeDict.__convert(v) for v in o)
        return o


a = SafeDict(dict(agh=1, bum=2, bar=3, baz=dict(bar=2,baz=dict(bar=3,baz=4))))
a["as"] = "if"
a.get("as"), type(a.get("baz")), a.get("baz")

In [ ]:
from pymbe.interpretation import *

def roll_up_upper_multiplicities(lpg):
    banded_featuring_graph = make_banded_featuring_graph(lpg)

    banded_roots = [
        banded_featuring_graph.nodes[node]
        for node in banded_featuring_graph
        if banded_featuring_graph.out_degree(node) == 0
    ]
    
    for part_usage in 

elements = lpg.elements_by_id

features = [
    id_
    for id_, data in elements.items()
    if data["@type"] == "Feature"
]

for feature in features:
    upper_multiplicity = get_feature_upper_multiplicity(elements, feature)
    if upper_multiplicity:
        print(elements[feature]["qualifiedName"], upper_multiplicity)

# Troubleshooting

## Fix linking issue with diagram element selector

In [ ]:
self = lpg.diagram

lpg.selected, self.selected, self.elk_app.diagram.selected

In [ ]:
self.id_mapper.get(*self.elk_app.diagram.selected)

In [ ]:
self._make_id_mapper()

In [ ]:
self = lpg

self._update_diagram_graph(button=self.filter_to_path)

In [ ]:
directed = self.path_directionality.value
new_graph = self._get_subgraph(
            excluded_edge_types=self.excluded_edge_types,
            excluded_node_types=self.excluded_node_types,
            reversed_edge_types=self.edge_type_reverser.value,
        )

source, target = self.selected
new_graph = self.get_path_graph(
    graph=new_graph,
    source=source,
    target=target,
    directed=directed,
)
len(new_graph)

In [ ]:
self.diagram.graph = new_graph

In [ ]:
import traitlets as trt


self.diagram._update_diagram(trt.Bunch(old=None))

In [ ]:
len(self.diagram.compound(self.diagram.container)[0])

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Mapper:
    to_map: dict = field(repr=False)
    from_map: dict = field(default=None, repr=False)

    def __repr__(self):
        return f"Mapper({len(self.to_map)} Items)"

    def __post_init__(self, *args, **kwargs):
        self.from_map = {v: k for k, v in self.to_map.items()}

    def get(self, *items):
        found = [
            self.to_map.get(item, self.from_map.get(item))
            for item in items
        ]
        return [
            item
            for item in found
            if item is not None
        ]
    
    def __len__(self):
        return len(self.to_map)

if Mapper({}):
    print("ahh")

---------------------

In [ ]:
self = lpg.diagram
import networkx as nx

_, hierarchy = self.elk_app.transformer.source

try:
    item = lpg.diagram.elk_app.selected[0]
    res = next(hierarchy.predecessors(item)).node
except nx.NetworkXError:
    res = item.data.get("properties", {}).get("@id")

res.id, res.data["@id"]

In [ ]:
tree.selected, explorer.selected, lpg.selected

In [ ]:
lpg.diagram.elk_app.diagram.selected

In [ ]:
lpg.selected

In [ ]:
self = lpg.diagram

_, hierarchy = self.elk_transformer.source

self.parts[self.selected[0]], self.elk_app.selected[0]

for selected_id in self.selected:
    node_selected = self.parts.get(selected_id)
    print(node_selected.id)

In [ ]:
node_selected.children[0] == self.elk_app.selected[0].node

---------------

In [ ]:
from dataclasses import dataclass, field


@dataclass
class Mapper:
    to_map: dict = field(repr=False)
    from_map: dict = field(default=None, repr=False)
        
    def __repr__(self):
        return f"Mapper({len(self.to_map)} Items)"

    def __post_init__(self, *args, **kwargs):
        self.from_map = {v: k for k, v in self.to_map.items()}

    def get(self, *items):
        found = [
            self.to_map.get(item, self.from_map.get(item))
            for item in items
        ]
        if len(found) < 2:
            found = found[0]
        return found

mapper = Mapper(from_elk_id)

mapper.get(*self.elk_app.diagram.selected)

In [ ]:
a = {1,2,}
a.symmetric_difference

In [ ]:
self = lpg.diagram

In [ ]:
relationships, hierarchy = self.elk_app.transformer.source

In [ ]:
transformer = self.elk_app.transformer
relationships, hierarchy = transformer.source

import ipyelk
from ipyelk.contrib.elements import (
    Compartment,
)

def id_from_item(item):
    id_ = None
    if isinstance(item, ipyelk.transform.Edge):
        id_ = item.data.get("properties", {}).get("@id")
    elif isinstance(getattr(item, "node", None), Compartment):
        id_ = next(hierarchy.predecessors(item)).node.id
    if id_ is None:
        self.log.debug(f"Could not parse: {item}")
    return id_

from_elk_id = {
    elk_id: id_from_item(elk_item)
    for elk_id, elk_item in transformer._elk_to_item.items()
}

from_elk_id = {
    elk_id: sysml_id
    for elk_id, sysml_id in from_elk_id.items()
    if sysml_id is not None
}
found_ids = set(from_elk_id.values())

for extra in set(lpg.elements_by_id).symmetric_difference(found_ids):
    print(lpg.elements_by_id[extra])




In [ ]:
transformer.from_id("61bd9433-3eec-47f7-af7e-3bf71f19071b").node

In [ ]:
missing = [elk_id for elk_id, our_id in from_elk_id.items() if our_id is None]
lpg.diagram.elk_app.diagram.selected = missing

In [ ]:
from_elk_id

# Scratch Pad
> **WARNING**: Anything below this point is just for experimentation purposes

## RDF Experiments

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation